In [ ]:
# this cell sets up the CSV reader to read data from disk

import os
import csv

def import_csv(headers: bool) -> list:
    # list of lists that will contain all tweet data
    results = []

    # makes list of all files in data directory
    csv_directory_path = os.path.join('data', 'inputs')
    csv_file_path = os.listdir(csv_directory_path)

    for file in csv_file_path:
        with open(os.path.join(csv_directory_path, file), mode='r') as csv_file:
            csv_reader = csv.reader(csv_file)
            if headers is True:
                next(csv_reader)
            for line in csv_reader:
                results.append(list(line))
    return results

We now need to initialize spaCy and load all of its dependencies.

In [14]:
import spacy

# imports the medium-sized English-language spaCy trained module, with vectors
nlp = spacy.load('en_core_web_lg')

### 2. Text Pre-processing

As the first step of our text pre-processing, we need to extract all of the named
entities from the tweet text. We will do this by first running the entire collected stream
through a spaCy pipeline.

We start by creating a list of only the tweet text data, then running that list through spaCy.

In [15]:
# we create a tuple of the data we want spaCy to ingest from the tweet_text and user_location fields
fetched_tweet_text = []
fetched_user_location = []
fetched_data_tuples = []
for _ in fetched_results:
    # list index of text tweet data within each tweet object
    fetched_tweet_text.append(_[2])
    fetched_user_location.append(_[-1])

# create a list of tuples of (tweet_text, user_location)
for text, location in zip(fetched_tweet_text, fetched_user_location):
    fetched_data_tuples.append((text, {'user_location': location}))

# creates a spaCy pipe, which processes input text data as a stream, returning a Doc object for each of those Docs
docs = list(nlp.pipe(fetched_data_tuples, as_tuples=True))

# prints out Doc data - only way to show context is to print during pipe creation.
# for doc, context in nlp.pipe(fetched_data_tuples, as_tuples=True):
#     print(doc.text)
#     print('\t', context)
#     for ent in doc.ents:
#         print('\t', ent.text, ent.label_)
#     print('---\n')
#     break

All of the processing on the Docs objects has already been done:: all that's left now is
to use the data.

We need to be able to process location data from tweets that don't contain it. Many will
have the data as part of the tweet text, but many won't. There are a few reasons for
this:

* context context
If a tweet is about a person, or about a well known event, location data is not
necessary, nor is it necessarily helpful.

* user context
If a tweet is from a small source, a local newspaper, or even a national source, often
the context is that the _source_ is local.

For the first reason, we must come up with ways to make sure that the context of the
tweet overrides the location data, even if it is provided. We need to come up with
ways of doing this, because of the steps we're going to take for the second reason.

For the second reason, we can inject the tweet user's location data if no other
location data exists in the tweet text.

In this code block, we try to complete the entire merging process, making the context Doc part of the tweet_text Doc.

In [16]:
def adding_user_location() -> list:
    """
    Uses the provided 'context' data added to a spaCy Doc, runs it through its
    own NLP pipeline to extract entity data, then returns a Doc object that
    contains NLP metadata to be added to the original Doc object it was derived from.

    param: (dict)

    returns: (spaCy Doc)
    """
    # list of Doc objects with context-added location data
    docs_with_gpe = []

    for doc in docs:
        ents = [(ent.text, ent.label_) for ent in doc[0].ents]
        # code block to determine of spaCy detected entities contain GPE
        contains_GPE = False
        if ents:
            for e in ents:
                if e[1] == "GPE":
                    contains_GPE = True
        if contains_GPE is False:
            doc_with_gpe = doc_reconstructor(doc)
            docs_with_gpe.append(doc_with_gpe)
        else:
            docs_with_gpe.append(doc[0])
    return docs_with_gpe


def doc_reconstructor(original_doc):
    """
    Takes the data from the old Doc (text and context) and combines it to make and return a new Doc.
    """
    combined_text_and_context = str(original_doc[0]) + '. ' + str(original_doc[1]['user_location']) + '.'
    doc = nlp(combined_text_and_context)
    return doc

docs_with_gpe = adding_user_location()

## 2. Pre-processing the Text

Here we have the option of extracting the tokens from each tweet text instance, or
extracting entire parts of speech from each of them.

In [17]:
# extracting tokens

def token_processor():
    # list that holds processed tokens in string form
    processed_docs = []

    for doc in docs_with_gpe:
        # creates list per doc
        doc_list = []
        # flag to determine if next token is a hashtag
        is_hashtag = False
        for token in doc:
            # removes hashtags by checking if the preceding token was a hashtag, assuming that
            # the next token would be the hashtag text; breaks from loop without adding to
            # processed token list if token is hashtag
            if token.text == '#':
                is_hashtag = True
            # checks if previous token was a hashtag character
            if is_hashtag is False:
                # checks if the token is an alpha character (removes numerals and punctuation)
                if token.is_alpha is True:
                    # checks if token is part of a stop list
                    if token.is_stop is False:
                        # checks if token is URL-like
                            if token.like_url is False:
                                # lowercases each token (uses the spaCy token's lowercase attribute)
                                token_text = token.lemma_
                                token_text_lemma = token_text.lower()
                                doc_list.append(token_text_lemma)
            # if is_hashtag has been set to True, skips processing logic and resets flag
            else:
                is_hashtag = False

        processed_docs.append(doc_list)

    return processed_docs

In [18]:
# extracting parts of speech
def part_of_speech_processor(docs: list) -> list:
    """
    The eventual idea is to parse out the five Ws: who, what, where, when, and why. This begins the
    process by starting to parse the text into its constituent parts of speech, starting a basic level.

    The output is a dictionary, with the following key:value pairs:
        text:   tweet text
        who:    any named entities that correspond with people or organizations
        what:   verbs and noun chunks that could correspond with events
        where:  any named entities that correspond with places (if none available, dafaults to tweet user location)
        when:   any named entities that correspond with time (if none availble, defaults to tweet timestamp)
        why:    TBD

    :param      docs: a list of spaCy Doc object containing a sequence of tokens and their linguistic annotations

    :returns    parts_of_speech: dictionary containing tweet text plus rule-based parsing of who, what, where, when, and why
    """

    # categorizes spaCy entity types (which themselves from from the OntoNotes 5 corpus: details at https://spacy.io/api/annotation#named-entities)
    who_types = ['NORP', 'ORG', 'PERSON']
    what_types = ['EVENT', 'LAW']
    where_types = ['GPE', 'LOC']
    when_types = ['DATE', 'TIME']
    why_types = []
    uncat_types = ['CARDINAL' 'FAC', 'LANGUAGE', 'MONEY', 'ORDINAL', 'PERCENT', 'PRODUCT', 'QUANTITY', 'WORK_OF_ART']

    parts_of_speech = []

    for doc in docs:
        who_tokens = []
        what_tokens = []
        where_tokens = []
        when_tokens = []
        why_tokens = []
        uncat_tokens = []

        noun_chunks = []

        ents = [(ent.text, ent.label_) for ent in doc.ents]
        if ents:
            for e in ents:
                if e[1] in who_types:
                    who_tokens.append(e[0])
                elif e[1] in what_types:
                    what_tokens.append(e[0])
                elif e[1] in where_types:
                    where_tokens.append(e[0])
                elif e[1] in when_types:
                    when_tokens.append(e[0])
                elif e[1] in why_types:
                    why_tokens.append(e[0])
                elif e[1] in uncat_types:
                    uncat_tokens.append([e[0], e[1]])
                else:
                    uncat_tokens.append(['UNKNOWN CATEGORY', e[0], e[1]])

        noun_chunks.append(list(doc.noun_chunks))

        parts_of_speech.append([doc, who_tokens, what_tokens, where_tokens, when_tokens, why_tokens,
                                uncat_tokens, noun_chunks])

    return parts_of_speech

In [19]:
extraction_type = 'pos'

if extraction_type == 'pos':
    summarized_tweets = part_of_speech_processor(docs_with_gpe)
elif extraction_type == 'token':
    summarized_tweets = token_processor()

## 3. Output

Since this is simply a testbed for using spaCy rules to summarize text inputs, what we have
now will be output to a CSV file for analysis and use in other applications.

The output CSV will be in `data/outputs`.

In [20]:
import datetime as dt

now_time = str(dt.datetime.now().strftime("%d-%m-%Y:%H:%M:%S"))

output_directory = os.path.join('data', 'outputs')
output_file = os.path.join(output_directory, now_time + '.csv')

with open(output_file, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['tweet_text', 'who_tokens', 'what_tokens', 'where_tokens', 'when_tokens', 'why_tokens',
                                'uncat_tokens', 'noun_chunks'])
    writer.writerows(summarized_tweets)